In [18]:

import cv2 as cv
import numpy as np
from utils.functions import *


def compute_homographies(video1_path, video2_path, video3_path, video4_path, frame_number1, frame_number2):
    def compute_H(frame1, frame2, x1_range=[0.4, 1.0], x2_range=[0.0, 0.7]):
        frame1, frame2 = undistort_and_rotate(frame1, dist_coeffs=np.array([-0.0733, 0.0833, 0, 0], dtype=np.float32), angle=-2.125), undistort_and_rotate(frame2, dist_coeffs=np.array([-0.0733, 0.0833, 0, 0], dtype=np.float32))
        frame1, frame2 = resize_to_equal_width(frame1, frame2)
        kp1, des1 = detect_and_compute_features(frame1, x_range=x1_range)
        kp2, des2 = detect_and_compute_features(frame2, x_range=x2_range)

        good_matches = match_features(des1, des2)
        H = compute_homography(kp1, kp2, good_matches)
        return H

    # 读取帧
    frame1, frame2 = read_frames_from_videos(video1_path, video2_path, frame_number1)
    # frame1 = resize_frame(frame1 , scale=0.98)
    frame3, frame4 = read_frames_from_videos(video3_path, video4_path, frame_number2)

    # 计算单应性矩阵
    H1 = compute_H(frame1, frame2)
    H2 = compute_H(frame3, frame4)

    sti1 = stitch_images_with_blending(frame1, frame2, H1)
    sti2 = stitch_images_with_blending(frame3, frame4, H2)

    H3 = compute_H(sti1, sti2, x1_range=[0.3, 1.0], x2_range=[0.0, 0.4])
    result = stitch_images_with_blending(sti1, sti2, H3)
    cv.imwrite("result.jpg", result)

    return H1, H2, H3

# 示例用法
video1_path = '../data/testolabc1.avi'
video2_path = '../data/testolabc2.avi'
video3_path = '../data/testolabc3.avi'
video4_path = '../data/testolabc4.avi'
frame_number1 = 0
frame_number2 = 890

H1, H2, H3 = compute_homographies(video1_path, video2_path, video3_path, video4_path, frame_number1, frame_number2)
# print(H1, H2, H3, sep='\n\n')

In [21]:
# 并行版本



import numpy as np
import os
from multiprocessing import Process, Lock
from time import time

def process_frames(start_frame, end_frame, lock, output_dir, start_time):
    video1_path = '../data/testolabc1.avi'
    video2_path = '../data/testolabc2.avi'
    video3_path = '../data/testolabc3.avi'
    video4_path = '../data/testolabc4.avi'
    
    for frame_number in range(start_frame, end_frame):
        frame1, frame2 = read_frames_from_videos(video1_path, video2_path, frame_number)
        frame3, frame4 = read_frames_from_videos(video3_path, video4_path, frame_number)

        frame1, frame2 = undistort_and_rotate(frame1, dist_coeffs=np.array([-0.0733, 0.0833, 0, 0], dtype=np.float32), angle=-2.125), undistort_and_rotate(frame2, dist_coeffs=np.array([-0.0733, 0.0833, 0, 0], dtype=np.float32))
        frame1, frame2 = resize_to_equal_width(frame1, frame2)
        frame3, frame4 = undistort_and_rotate(frame3, dist_coeffs=np.array([-0.0733, 0.0833, 0, 0], dtype=np.float32), angle=-2.125), undistort_and_rotate(frame4, dist_coeffs=np.array([-0.0733, 0.0833, 0, 0], dtype=np.float32))
        frame3, frame4 = resize_to_equal_width(frame3, frame4)

        # H1, H2, H3 是预先计算好的全局变量
        sti1, sti2 = stitch_images_with_blending(frame1, frame2, H1), stitch_images_with_blending(frame3, frame4, H2)
        result = stitch_images_with_blending(sti1, sti2, H3)

        elapsed_time = time() - start_time
        frame_rate = num_processes * frame_number / elapsed_time if elapsed_time > 0 else 0
        text = f"FPS: {frame_rate:.2f} Time: {elapsed_time:.2f}"

        cv.putText(result, text, (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        with lock:
            output_path = os.path.join(output_dir, f'{frame_number}.jpg')
            cv.imwrite(output_path, result)

if __name__ == "__main__":
    start_time = time()
    output_dir = 'output'
    os.makedirs(output_dir, exist_ok=True)
    
    num_processes = 8  # 设置进程数量
    total_frames = 800
    frames_per_process = total_frames // num_processes
    lock = Lock()
    processes = []

    # 启动多个进程
    for i in range(num_processes):
        start_frame = i * frames_per_process
        end_frame = (i + 1) * frames_per_process if i != num_processes - 1 else total_frames
        p = Process(target=process_frames, args=(start_frame, end_frame, lock, output_dir, start_time))
        processes.append(p)
        p.start()

    # 等待所有进程完成
    for p in processes:
        p.join()

    print("所有帧处理完毕。")

 

 


所有帧处理完毕。


In [29]:


def images_to_video(image_folder, output_video_path, fps=30):
    # 获取所有图片文件名并排序
    images = [img for img in os.listdir(image_folder) if img.endswith((".png", ".jpg", ".jpeg"))]
    images.sort()

    # 读取第一张图片以获取帧的宽度和高度
    frame = cv.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    # 定义视频编写器
    fourcc = cv.VideoWriter_fourcc(*'mp4v')  # 使用mp4v编码
    video = cv.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # 将每张图片写入视频
    for image in images:
        img_path = os.path.join(image_folder, image)
        frame = cv.imread(img_path)
        video.write(frame)

    # 释放视频编写器
    video.release()
    print(f"视频保存到 {output_video_path}")

# 示例用法
image_folder = 'output'
output_video_path = 'output_video.mp4'
images_to_video(image_folder, output_video_path, fps=30)



视频保存到 output_video.mp4
